In [32]:
import sqlite3
con = sqlite3.connect("products.db")
cur = con.cursor()

In [33]:
cur.execute("DROP TABLE IF EXISTS Warehouses")
cur.execute("DROP TABLE IF EXISTS Boxes")

In [34]:
cur.executescript("""
CREATE TABLE Warehouses (
   Code INTEGER NOT NULL,
   Location VARCHAR(255) NOT NULL ,
   Capacity INTEGER NOT NULL,
   PRIMARY KEY (Code)
 );
CREATE TABLE Boxes (
    Code CHAR(4) NOT NULL,
    Contents VARCHAR(255) NOT NULL ,
    Value REAL NOT NULL ,
    Warehouse INTEGER NOT NULL,
    PRIMARY KEY (Code)
 );

 INSERT INTO Warehouses(Code,Location,Capacity) VALUES(1,'Chicago',3);
 INSERT INTO Warehouses(Code,Location,Capacity) VALUES(2,'Chicago',4);
 INSERT INTO Warehouses(Code,Location,Capacity) VALUES(3,'New York',7);
 INSERT INTO Warehouses(Code,Location,Capacity) VALUES(4,'Los Angeles',2);
 INSERT INTO Warehouses(Code,Location,Capacity) VALUES(5,'San Francisco',8);

 INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('0MN7','Rocks',180,3);
 INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('4H8P','Rocks',250,1);
 INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('4RT3','Scissors',190,4);
 INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('7G3H','Rocks',200,1);
 INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('8JN6','Papers',75,1);
 INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('8Y6U','Papers',50,3);
 INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('9J6F','Papers',175,2);
 INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('LL08','Rocks',140,4);
 INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('P0H6','Scissors',125,1);
 INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('P2T6','Scissors',150,2);
 INSERT INTO Boxes(Code,Contents,Value,Warehouse) VALUES('TU55','Papers',90,5);

""")
con.commit()

In [35]:
# --3.1 Select all warehouses.
cur.execute("SELECT * FROM Warehouses")
display(cur.fetchall())

[(1, 'Chicago', 3),
 (2, 'Chicago', 4),
 (3, 'New York', 7),
 (4, 'Los Angeles', 2),
 (5, 'San Francisco', 8)]

In [36]:
# --3.2 Select all boxes with a value larger than $150.
cur.execute("SELECT * FROM Boxes WHERE Value > 150")
display(cur.fetchall())

[('0MN7', 'Rocks', 180.0, 3),
 ('4H8P', 'Rocks', 250.0, 1),
 ('4RT3', 'Scissors', 190.0, 4),
 ('7G3H', 'Rocks', 200.0, 1),
 ('9J6F', 'Papers', 175.0, 2)]

In [37]:
# --3.3 Select all distinct contents in all the boxes.
cur.execute("SELECT DISTINCT Contents FROM Boxes")
display(cur.fetchall())

[('Rocks',), ('Scissors',), ('Papers',)]

In [38]:
# --3.4 Select the average value of all the boxes.
cur.execute("SELECT AVG(Value) FROM Boxes")

In [39]:
# --3.5 Select the warehouse code and the average value of the boxes in each warehouse.
cur.execute("""
    SELECT Warehouse, AVG(Value)
    FROM Boxes
    GROUP BY Warehouse
""")
display(cur.fetchall())

[(1, 162.5), (2, 162.5), (3, 115.0), (4, 165.0), (5, 90.0)]

In [40]:
# --3.6 Same as previous question, but select only those warehouses where the average value of the boxes is greater than $150.
cur.execute("""
    SELECT Warehouse, AVG(Value)
    FROM Boxes
    GROUP BY Warehouse
    HAVING AVG(Value) > 150
""")
display(cur.fetchall())

[(1, 162.5), (2, 162.5), (4, 165.0)]

In [41]:
# --3.7 Select the code of each box, along with the name of the city the box is located in.
cur.execute("""
    SELECT Boxes.Code, Warehouses.Location
    FROM Boxes
    INNER JOIN Warehouses
    ON Boxes.Warehouse = Warehouses.Code
""")
display(cur.fetchall())

[('0MN7', 'New York'),
 ('4H8P', 'Chicago'),
 ('4RT3', 'Los Angeles'),
 ('7G3H', 'Chicago'),
 ('8JN6', 'Chicago'),
 ('8Y6U', 'New York'),
 ('9J6F', 'Chicago'),
 ('LL08', 'Los Angeles'),
 ('P0H6', 'Chicago'),
 ('P2T6', 'Chicago'),
 ('TU55', 'San Francisco')]

In [42]:
# --3.8 Select the warehouse code, and the average value of the boxes in each warehouse, only for those warehouses where the average value of the boxes is greater than $150.
cur.execute("""
    SELECT Warehouse, AVG(Value)
    FROM Boxes
    GROUP BY Warehouse
    HAVING AVG(Value) > 150
""")
display(cur.fetchall())

[(1, 162.5), (2, 162.5), (4, 165.0)]

In [43]:
# --3.9 Select the codes of all warehouses that are saturated (a warehouse is saturated if the number of boxes in it is larger than the warehouse's capacity).
cur.execute("""
    SELECT Boxes.Warehouse
    FROM Boxes
    JOIN Warehouses ON Boxes.Warehouse = Warehouses.Code
    GROUP BY Boxes.Warehouse
    HAVING COUNT(Boxes.Warehouse) > Warehouses.Capacity
""")
display(cur.fetchall())

[(1,)]

In [44]:
# --3.10 Select the codes of all the boxes located in Chicago.
cur.execute("""
    SELECT Boxes.Code
    FROM Boxes
    JOIN Warehouses ON Boxes.Warehouse = Warehouses.Code
    WHERE Warehouses.Location = 'Chicago'
""")
display(cur.fetchall())

[('4H8P',), ('7G3H',), ('8JN6',), ('9J6F',), ('P0H6',), ('P2T6',)]

In [45]:
# --3.11 Create a new warehouse in New York with a capacity for 3 boxes.
cur.execute("""
    INSERT INTO Warehouses(Code,Location,Capacity)
    VALUES(6,'New York',3)
""")
con.commit()

In [46]:
# --3.12 Create a new box, with code "H5RT", containing "Papers" with a value of $200, and located in warehouse 2.
cur.execute("""
    INSERT INTO Boxes(Code,Contents,Value,Warehouse)
    VALUES('H5RT','Papers',200,2)
""")
con.commit()

In [47]:
# --3.13 Reduce the value of all boxes by 15%.
cur.execute("""
    UPDATE Boxes
    SET Value = Value * 0.85
""")
con.commit()

In [48]:
# --3.14 Remove all boxes with a value lower than $100.
cur.execute("""
    DELETE FROM Boxes
    WHERE Value < 100
""")
con.commit()

In [49]:
# --3.15 Remove all boxes from saturated warehouses.
cur.execute("""
    DELETE FROM Boxes
    WHERE Warehouse IN (
        SELECT Boxes.Warehouse
        FROM Boxes
        JOIN Warehouses ON Boxes.Warehouse = Warehouses.Code
        GROUP BY Boxes.Warehouse
        HAVING COUNT(Boxes.Warehouse) > Warehouses.Capacity
    )
""")
con.commit()

In [50]:
# -- 3.16 Add Index for column "Warehouse" in table "boxes"
cur.execute("""
    CREATE INDEX idx_warehouse ON Boxes(Warehouse)
""")
con.commit()

In [51]:
# -- 3.17 Print all the existing indexes
cur.execute("""
    SELECT name FROM sqlite_master WHERE type='index'
""")

In [52]:
# -- 3.18 Remove Index for column "Warehouse" in table "boxes"
cur.execute("""
    DROP INDEX idx_warehouse
""")
con.commit()